# Python Environments in VS Code
https://code.visualstudio.com/docs/python/environments

Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope Process \
.\.venv\Scripts\Activate.ps1

# Environmental Varialbles
Setup the environmental variables below. \
INFLUXDB3_AUTH_TOKEN="<>" \
INFLUXDB_V2_URL="<>" \
INFLUXDB_V2_ORG="<>" \
INFLUXDB_V2_TOKEN="<>"

# Python Libraries 
python -m pip install --upgrade pip \
python -m pip install pandas[performance] \
python -m pip install influxdb3-python \
python -m pip install 'influxdb-client[ciso]'
python -m pip install matplotlib \
python -m pip install bokeh \
python -m pip install seaborn \
python -m pip install scipy \
python -m pip install plotly \
python -m pip install nbformat

### Windows
You have to install Visual C++ Build Tools 2015 to build ciso8601 by pip. \
https://github.com/influxdata/influxdb-client-python

# Python Code
Using Python 3.13.4
## Initialize an InfluxDB 3 client.

In [ ]:
import os
import pandas as pd
from influxdb_client_3 import InfluxDBClient3

# InfluxDB3 Access Information
DATABASE = "FEATHER_WEATHER"
AUTH_TOKEN = os.getenv('INFLUXDB3_AUTH_TOKEN')
# AUTH_TOKEN = "<INFLUXDB3_AUTH_TOKEN>"
print(f"AUTHORIZATION TOKEN: {AUTH_TOKEN}") # Add an error and quit if this is zero or None.
HOST = "http://192.168.50.138:8181"

client = InfluxDBClient3(host=HOST,
                         database=DATABASE,
                         token=AUTH_TOKEN)


## Close the InfluxDB 3 Client

In [ ]:
client.close()

## Show Tables

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"

QUERY = "SHOW TABLES"

table = client.query(query=QUERY, language="sql", mode="all")
print(table)
print()
table = client.query(query=QUERY, language="sql", mode="pandas")
print(table)
print()
display(table)

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"

QUERY = "SHOW COLUMNS FROM weather"

table = client.query(QUERY)
print(table)
print()

table = client.query(query=QUERY, language="sql", mode="pandas")
print(table)
print()
display(table)

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"

QUERY = "SELECT * FROM weather WHERE time >= now() - INTERVAL '5 minutes'"

table = client.query(query=QUERY, language="sql", mode="pandas")
print(table)
print()
display(table)

fig1 = px.line(table, x="time", y="temperature", color="node", title="Temperature Over Time")
fig1.show()

In [ ]:
table = client.query("SHOW COLUMNS IN home")
print(table)

In [ ]:
table = client.query("SELECT * from home WHERE time >= now() - INTERVAL '4 years'")

# Filter columns.
print(table.select(['room', 'temp']))

# Use PyArrow to aggregate data.
print(table.group_by('hum').aggregate([]))

In [ ]:
QUERY = "SELECT * FROM weather WHERE time >= now() - INTERVAL '5 minutes'"

table = client.query(query=QUERY, language="sql", mode="all")

# Filter columns.
print(table.select(['node', 'relative_humidity']))

# Use PyArrow to aggregate data.
print(table.group_by('temperature').aggregate([]))

In [ ]:
QUERY = "SELECT * FROM weather WHERE time >= now() - INTERVAL '5 minutes'"
table = client.query(query=QUERY, language="sql", mode="all")
print(table.to_pandas())
print()
df = table.sort_by("time")
print(df)

## PyArrow Aggregation

In [ ]:
QUERY = "SELECT * FROM weather WHERE time >= now() - INTERVAL '24 hours'"
table = client.query(query=QUERY, language="sql", mode="all")
aggregation_temperature = table.group_by("node").aggregate([("temperature", "mean"),
                                                ("temperature", "max"),
                                                ("temperature", "min")]).to_pandas()
print(aggregation_temperature)
print()

aggregation_humidity = table.group_by("node").aggregate([("relative_humidity", "mean"),
                                                ("relative_humidity", "max"),
                                                ("relative_humidity", "min")]).to_pandas()
print(aggregation_humidity)
print()

aggregation_pressure = table.group_by("node").aggregate([("pressure", "mean"),
                                                ("pressure", "max"),
                                                ("pressure", "min")]).to_pandas()
print(aggregation_pressure)
print()

aggregation_co2 = table.group_by("node").aggregate([("co2", "mean"),
                                                ("co2", "max"),
                                                ("co2", "min")]).to_pandas()
print(aggregation_co2)

# InfluxDB 2
## Initialize an InfluxDB 2 Client

In [ ]:
import os
import influxdb_client
from influxdb_client import InfluxDBClient, Point, Dialect

# InfluxDB Access Information
# INFLUXDB_V2_URL = "<>"
# INFLUXDB_V2_ORG = "<>"
# INFLUXDB_V2_TOKEN = "<>"
TOKEN = os.getenv('INFLUXDB_V2_TOKEN')
# print(f"AUTHORIZATION TOKEN: {TOKEN}")
ORG = "databyBen"
BUCKET = "Environmental Metrics"
URL = "http://rpi-2.local:8086"

client_v2 = influxdb_client.InfluxDBClient(
    url=URL, 
    token=TOKEN, 
    org=ORG
    )

# write_api = client_v2.write_api(write_options=SYNCHRONOUS)
query_api = client_v2.query_api()

ready_status = client_v2.ready()  # Readiness of InfluxDB 2.0
print(ready_status)

## Close the InfluxDB 2 Client

In [ ]:
client_v2.close()

## Query InfluxDB 2

In [ ]:
QUERY = """from(bucket: "Environmental Metrics")
    |> range(start: -5m)
    |> filter(fn: (r) => r["_measurement"] == "weather")
    |> filter(fn: (r) => r["_field"] == "temperature")
    |> filter(fn: (r) => r["node"] == "sht40")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")"""

table_v2 = query_api.query(query=QUERY, org=ORG)

print(table_v2)

## CSV Query

In [ ]:
QUERY = """from(bucket: "Environmental Metrics")
    |> range(start: -5m)
    |> filter(fn: (r) => r["_measurement"] == "weather")
    |> filter(fn: (r) => r["_field"] == "temperature")
    |> filter(fn: (r) => r["node"] == "sht40")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")"""

table_v2 = query_api.query(query=QUERY, org=ORG)

print(f"Use the number of columns and records for the for loop below. \n {table_v2}\n")

csv_result = query_api.query_csv(query=QUERY, dialect=Dialect(header=False, delimiter=",", comment_prefix="#", annotations=[], date_time_format="RFC3339"))

print(csv_result)

for csv_line in csv_result:
    if not len(csv_line) == 0:
        print(f'Temperature in {csv_line[8]} is {csv_line[5]}')

"""
Close client
"""
# client.close()


## Pandas Data Frame

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"

QUERY = """from(bucket: "Environmental Metrics")
    |> range(start: -7d)
    |> filter(fn: (r) => r["_measurement"] == "weather")
    |> filter(fn: (r) => r["_field"] == "temperature")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")"""

df = query_api.query_data_frame(query=QUERY, org=ORG)

print(df)
print()
display(df)

fig1 = px.line(df, x="_time", y="temperature", color="node", title="Temperature Over Time")
fig1.show()

In [ ]:
""" June 5, 2025
Runs on Python 3.11.2 on Raspberry Pi OS.
Runs on Python 2.12.10 on Windows 11.
Runs on Python 3.13.4 on Windows 11. """

import os, time
import influxdb_client
# from influxdb_client import InfluxDBClient, Point, WritePrecision
# from influxdb_client.client.write_api import SYNCHRONOUS

# bucket = "<my-bucket>"
# token = os.environ.get("INFLUXDB_TOKEN") # export INFLUXDB_TOKEN=<my-token>
TOKEN = os.getenv('INFLUXDB_V2_TOKEN')
# print(f"AUTHORIZATION TOKEN: {TOKEN}")
ORG = "databyBen"
URL = "http://rpi-2.local:8086"

# write_client = influxdb_client.InfluxDBClient(url=URL, token=token, org=ORG)

client = influxdb_client.InfluxDBClient(
   url=URL,
   token=TOKEN,
   org=ORG
)

query_api = client.query_api()

QUERY = """from(bucket: "Environmental Metrics")
  |> range(start: -5m)
  |> filter(fn: (r) => r["_measurement"] == "weather")
  |> filter(fn: (r) => r["node"] == "scd4x")"""
tables = query_api.query(QUERY, org=ORG)

for table in tables:
    for record in table.records:
        print(record)

client.close()
